<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.2.3 
# *Mining Social Media on Reddit*

## The Reddit API and the PRAW Package

The Reddit API is rich and complex, with many endpoints (https://www.reddit.com/dev/api/). It includes methods for navigating its collections, which include various kinds of media as well as comments. Fortunately, the Python library PRAW reduces much of this complexity.

Reddit requires developers to create and authenticate an app before they can use the API, but the process is much less onerus than some, and does not have waiting period for approval of new developers (as of 18 August 2018).

### 1. Create a Reddit App

Go to https://www.reddit.com/prefs/apps and click "create an app".

Enter the following in the form:

- a name for your app
- select "script" radio button
- a description
- a redirect URI

(Nb. For pulling data into a data science experiment, a local port can be used for the Redirect URI; try http://127.0.0.1:1410)


- click "create app"
- from the form that displays, copy the following to a local text file (or to this notebook):

  - name (the name you gave to your app)
  - redirect URI
  - personal use script (this is your OAuth 2 Client ID)
  - secret (this is your OAuth 2 Secret)

### 2. Register for API Access

- follow the link at https://www.reddit.com/wiki/api and read the terms of use for Reddit API access 
- fill in the form fields at the bottom 
  - make sure to enter your new OAuth Client ID where indicated
  - your use case could be something like "Training in API usage for data science projects"
  - your platform could be something like "Jupyter Notebooks / Python"
  
- click "SUBMIT"
 
- when asked for User-Agent, enter something that fits this pattern:
  `your_os-python:your_reddit_appname:v1.0 (by /u/your_reddit_username)`

### 3. Load Python Libraries

In [1]:
import praw
import requests
import json
import pprint
from datetime import datetime, date, time

### 4. Authenticate from your Python script

You could assign your authentication details explicitly, as follows:

In [2]:
##here we have access to our pw
with open('projectscrapepeewee.txt', 'r') as f:
    pw = f.read()

In [3]:
my_user_agent = 'autist bot v1.0 by /u/trdr1988'   # your user Agent string goes in here
my_client_id = '6tsVwaR0qLinZg'   # your Client ID string goes in here
my_client_secret = pw   # your Secret string goes in here

A better way would be to store these details externally, so they are not displayed in the notebook:

- create a file called "auth_reddit.json" in your "notebooks" directory, and save your credentials there in JSON format:

`{   "my_client_id": "your Client ID string goes in here",` <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;` "my_client_secret": "your Secret string goes in here",` <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`"my_user_agent": "your user Agent string goes in here"` <br>
`}`

Use the following code to load the credentials:  

In [34]:
credentials = 'creds.json'
with open(credentials) as f:
    creds=json.load(f)

In [0]:
#second method of 0auth. not necessery
pwd()  # make sure your working directory is where the file is

In [20]:
#second method of 0auth. not necessery
path_auth = 'creds.json'
auth = json.loads(open(path_auth).read())
pp = pprint.PrettyPrinter(indent=4)
# For debugging only:
#pp.pprint(auth)

my_user_agent = auth['my_user_agent']
my_client_id = auth['my_client_id']
my_client_secret = auth['my_client_secret']

Security considerations: 
- this method only keeps your credentials invisible as long as nobody else gets access to this notebook file 
- if you wanted another user to have access to the executable notebook without divulging your credentials you should set up an OAuth 2.0 workflow to let them obtain and apply their own API tokens when using your app
- if you just want to share your analyses, you could use a separate script (which you don't share) to fetch the data and save it locally, then use a second notebook (with no API access) to load and analyse the locally stored data

### 5. Exploring the API

Here is how to connect to Reddit with read-only access:

In [4]:
reddit = praw.Reddit(client_id = my_client_id, 
                     client_secret = my_client_secret, 
                     user_agent = my_user_agent)

print('Read-only = ' + str(reddit.read_only))  # Output: True

Read-only = True


Version 7.1.0 of praw is outdated. Version 7.3.0 was released 7 days ago.


In the next cell, put the cursor after the '.' and hit the [tab] key to see the available members and methods in the response object:

In [5]:
reddit.info

<bound method Reddit.info of <praw.reddit.Reddit object at 0x7f945b43bf50>>

Consult the PRAW and Reddit API documentation. Print a few of the response members below:

In [6]:
print(reddit.comment)
print(reddit.domain)

<bound method Reddit.comment of <praw.reddit.Reddit object at 0x7f945b43bf50>>
<bound method Reddit.domain of <praw.reddit.Reddit object at 0x7f945b43bf50>>


Content in Reddit is grouped by topics called "subreddits". Content, called "submissions", is fetched by calling the `subreddit` method of the connection object (which is our `reddit` variable) with an argument that matches an actual topic. 

We also need to append a further method call to a "subinstance", such as one of the following:

- controversial
- gilded
- hot
- new
- rising
- top

One of the submission objects members is `title`. Fetch and print 10 submission titles from the 'learnpython' subreddit using one of the subinstances above:

In [7]:
for submission in reddit.subreddit('wallstreetbets').hot(limit=10):
    print(submission.title)
    print('=================')

What Are Your Moves Tomorrow, June 25, 2021
Fraternal Association of Gambling Gentlemen and Yacht Degenerates for June 24, 2021
If Tesla manages to hit 750+ by the end of the week, I will make a video of me drinking coffee and post it here. (I am an active member of the LDS church, otherwise known as a Mormon)
$PSFE Power up your wallet
Did my first bit of simple DD on Paysafe (PSFE)
$WISH me luck
Biden Says He Has Infrastructure Deal With Senate Group. HRC futures hit $1800 ATH. My 117,099 CLF shares are having a steel erection
$WISH in honor of u/Unusual-Obligation-7 YOLO
I analyzed the last 3 decades of stock market returns to determine if it makes sense to time the market! Here are the results!
4MM Yolo $WISH, Caesar is home, where are my ape brothers and sisters


In [ ]:
#from above we can see that we want the first and second threads always

Now retrieve 10 authors:

In [8]:
for submission in reddit.subreddit('wallstreetbets').hot(limit=10):
    print(submission.author)

OPINION_IS_UNPOPULAR
VisualMod
SnooRegrets7769
sultanmirza007
Kleeneks
EstablishmentSome283
SIR_JACK_A_LOT
Own_Chocolate_5006
nobjos
Unusual-Obligation-7


Note that we obtained the titles and authors from separate API calls. Can we expect these to correspond to the same submissions? If not, how could we gurantee that they do?

In [31]:
#fetch the url as well so you can crosscheck
for submission in reddit.subreddit('wallstreetbets').hot(limit=10):
    print(submission.title)
    print(submission.author)
    print(submission.url)
    print('==============')

Daily Discussion Thread for June 22, 2021
OPINION_IS_UNPOPULAR
https://www.reddit.com/r/wallstreetbets/comments/o5jefj/daily_discussion_thread_for_june_22_2021/
Daily Popular Tickers Thread for June 22, 2021 - AMC | GME | BB
VisualMod
https://www.reddit.com/r/wallstreetbets/comments/o5jeql/daily_popular_tickers_thread_for_june_22_2021_amc/
Hedge fund that bet against GameStop is closing down
Illustrious_Welder94
https://www.reddit.com/r/wallstreetbets/comments/o5f72j/hedge_fund_that_bet_against_gamestop_is_closing/
GameStop Completes At-The-Market Equity Offering Program
quantkim
https://investor.gamestop.com/news-releases/news-release-details/gamestop-completes-market-equity-offering-program-0
What’s changed for me since joining WSB.
Huhlol55555
https://v.redd.it/6t923htsro671
GME yolo. Tryna get it while the gettins good
veryeducatedinvestor
https://i.redd.it/drc9py5qjr671.jpg
How I’ve felt since joining WSB back in January
-crentist-
https://v.redd.it/vom0jeljcp671
Got me a first cl

In [11]:
#fetch the url as well so you can crosscheck
for submission in reddit.subreddit('wallstreetbets').hot(limit=10):
    top_level_comments = list(submission.comments)
    all_comments = submission.comments.list()[:2]
    for comment in all_comments:
        print(comment.body)
        print(comment.author)

My WISH was secured, gracias
SilkyThighs
I will group text my parents a video of my girlfriend pegging me tomorrow if TSLA opens at $695. Mods can hold me to it.
AstroOhio

**User Report**| | | |
:--|:--|:--|:--
**Total Submissions**|33|**First Seen In WSB**|6 months ago
**Total Comments**|72079|**Previous DD**|
**Account Age**|1 year|[^scan ^comment ](https://www.reddit.com/message/compose/?to=VisualMod&subject=scan_comment&message=Replace%20this%20text%20with%20a%20comment%20ID%20(which%20looks%20like%20*h26cq3k*)%20to%20have%20the%20bot%20scan%20your%20comment%20and%20correct%20your%20first%20seen%20date.)|[^scan ^submission ](https://www.reddit.com/message/compose/?to=VisualMod&subject=scan_submission&message=Replace%20this%20text%20with%20a%20submission%20ID%20(which%20looks%20like%20*h26cq3k*)%20to%20have%20the%20bot%20scan%20your%20submission%20and%20correct%20your%20first%20seen%20date.)
VisualMod
I see why you like FDs. These leaps are clogging up my port.

Fuck me or make me 


**User Report**| | | |
:--|:--|:--|:--
**Total Submissions**|1|**First Seen In WSB**|just now
**Total Comments**|0|**Previous DD**|
**Account Age**|11 months|[^scan ^comment ](https://www.reddit.com/message/compose/?to=VisualMod&subject=scan_comment&message=Replace%20this%20text%20with%20a%20comment%20ID%20(which%20looks%20like%20*h26cq3k*)%20to%20have%20the%20bot%20scan%20your%20comment%20and%20correct%20your%20first%20seen%20date.)|[^scan ^submission ](https://www.reddit.com/message/compose/?to=VisualMod&subject=scan_submission&message=Replace%20this%20text%20with%20a%20submission%20ID%20(which%20looks%20like%20*h26cq3k*)%20to%20have%20the%20bot%20scan%20your%20submission%20and%20correct%20your%20first%20seen%20date.)
VisualMod
Anyone else find it heartbreaking to watch this scene when the show was peaking now that we know it just death spirals in the last season
slicePuff

**User Report**| | | |
:--|:--|:--|:--
**Total Submissions**|46|**First Seen In WSB**|5 months ago
**Total Com

Why doesn't the next cell produce output?

In [13]:
for submission in submissions:
    print(submission.comments)

NameError: name 'submissions' is not defined

A:"submissions" is not the variable created to access the reddit API, a connection to the API and subreddit must be picked. create a variable called submissions to loop through

Print two comments associated with each of these submissions:

In [9]:
submissions = reddit.subreddit('wallstreetbets').hot(limit=10)
for submission in submissions:
    top_level_comments = list(submission.comments)
    all_comments = submission.comments.list()[:2]
    for comment in all_comments:
        print(comment.body)

Bought 5 WISH calls at 14.5 right before close 🥰
I loaded up or YOLO on CLOV with 6k USD today, likely to buy more shares tomorrow.

Will buy more WISH.

Will load up on WKHS when fresh money arrives 

so WISH CLOV WKHS will rise to the stars 🚀🚀🚀🚀
We are pinning this thread to the frontpage as an experiment.

As fanny put it the other day, this thread is not designed to be a popularity contest or echo chamber. This is a place for vetted WSBettors to share thoughts on trades / tickers without just posting into the oblivion of high-traffic threads. If you see users posting smart takes on stocks, please induct them. Keep in mind though this has implications beyond this thread, as it means they will be ignored by most of our anti-spam rules across the sub.
I see why you like FDs. These leaps are clogging up my port.

Fuck me or make me rich and gtfo. I am not a classy woman.

**User Report**| | | |
:--|:--|:--|:--
**Total Submissions**|18|**First Seen In WSB**|2 weeks ago
**Total Comments*


**User Report**| | | |
:--|:--|:--|:--
**Total Submissions**|1|**First Seen In WSB**|just now
**Total Comments**|0|**Previous DD**|
**Account Age**|11 months|[^scan ^comment ](https://www.reddit.com/message/compose/?to=VisualMod&subject=scan_comment&message=Replace%20this%20text%20with%20a%20comment%20ID%20(which%20looks%20like%20*h26cq3k*)%20to%20have%20the%20bot%20scan%20your%20comment%20and%20correct%20your%20first%20seen%20date.)|[^scan ^submission ](https://www.reddit.com/message/compose/?to=VisualMod&subject=scan_submission&message=Replace%20this%20text%20with%20a%20submission%20ID%20(which%20looks%20like%20*h26cq3k*)%20to%20have%20the%20bot%20scan%20your%20submission%20and%20correct%20your%20first%20seen%20date.)
Anyone else find it heartbreaking to watch this scene when the show was peaking now that we know it just death spirals in the last season

**User Report**| | | |
:--|:--|:--|:--
**Total Submissions**|46|**First Seen In WSB**|5 months ago
**Total Comments**|145|**Previo

Referring to the API documentation, explore the submissions object and print some interesting data:

In [ ]:
#LETS TRY PUTTING IT INTO A DB

In [45]:
file_r = open("db.csv", 'r')
lines = file_r.readlines()
file_w = open("db.csv", 'w')
for line in lines:
    file_w.write(line)

In [46]:
for submission in reddit.subreddit('wallstreetbets').search("title:'searchtext'",sort='new',limit=1000):
    file_w.write(submission.id+','+submission.title+','+submission.selftext+'\n')

ResponseException: received 401 HTTP response

#### Posting to Reddit

To be able to post to your Reddit account (i.e. contribute submissions), you need to connect to the API with read/write privilege. This requires an *authorised instance*, which is obtained by including your Reddit user name and password in the connection request: 

In [35]:
reddit = praw.Reddit(client_id='my client id',
                     client_secret='my client secret',
                     user_agent='my user agent',
                     username='my username',
                     password='my password')
print(reddit.read_only)  # Output: False

False


In [32]:
#trial 1
subr = 'devs' # Choose your subreddit
 
subreddit = reddit.subreddit(subr) # Initialize the subreddit to a variable
 
title = 'Just Made My first Post on Reddit Using Python.'
 
selftext = '''
I am learning how to use the Reddit API with Python using the PRAW wrapper.
Give me some karma.
This post was uploaded from my Python Script
'''

In [ ]:
subreddit.submit(title,selftext=selftext)

You could hide these last two credentials by adding them to your JSON file and then reading all five values at once.

In [31]:
subreddit.submit(title,selftext=selftext)

/Users/lawrenceyim/opt/anaconda3/envs/api/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Reddit will check for validation on all posts around May-June 2020. It is recommended to check for validation by setting reddit.validate_on_submit to True.
  """Entry point for launching an IPython kernel.


ResponseException: received 401 HTTP response

>
>


>
>




---



---



> > > > > > > > > © 2021 Institute of Data


---



---



